# ANLI - GloVE

https://towardsdatascience.com/glove-elmo-bert-9dbbc9226934

In [1]:
pip install -q torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q pyspark==3.3.0 spark-nlp==4.3.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
johnsnowlabs 4.4.4 requires pyspark==3.1.2, but you have pyspark 3.3.0 which is incompatible.
johnsnowlabs 4.4.4 requires spark-nlp==4.4.1, but you have spark-nlp 4.3.2 which is incompatible.
flair 0.11.1 requires more-itertools~=8.8.0, but you have more-itertools 8.12.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from torchtext.vocab import GloVe
import pandas as pd

In [4]:
import pyspark
import sparknlp

In [5]:
# load datasets and convert them into dataframes
dataset = load_dataset('anli')
train_r1 = pd.DataFrame(dataset['train_r1'])
dev_r1 = pd.DataFrame(dataset['dev_r1'])
test_r1 = pd.DataFrame(dataset['test_r1'])
train_r2 = pd.DataFrame(dataset['train_r1'])
dev_r2 = pd.DataFrame(dataset['dev_r2'])
test_r2 = pd.DataFrame(dataset['test_r2'])
train_r3 = pd.DataFrame(dataset['train_r3'])
dev_r3 = pd.DataFrame(dataset['dev_r3'])
test_r3 = pd.DataFrame(dataset['test_r3'])

Found cached dataset anli (/Users/sofiamurillosanchez/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)


  0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
def convert_to_glove(df):
    new_df = {'label': df['label'], 'text': df['hypothesis'] + df['premise']}
    return pd.DataFrame(data=new_df)

In [7]:
glove_train_r1 = convert_to_glove(train_r1)
glove_dev_r1 = convert_to_glove(dev_r1)
glove_test_r1 = convert_to_glove(test_r1)

glove_train_r2 = convert_to_glove(train_r2)
glove_dev_r2 = convert_to_glove(dev_r2)
glove_test_r2 = convert_to_glove(test_r2)

glove_train_r3 = convert_to_glove(train_r3)
glove_dev_r3 = convert_to_glove(dev_r3)
glove_test_r3 = convert_to_glove(test_r3)
glove_train_r1

,label,text
0,0,The trolleybus system has over 2 urban routesT...
1,1,Sharron Macready was a popular character throu...
2,1,Bastedo didn't keep any pets because of her vi...
3,1,Alexandra Bastedo was named by her mother.Alex...
4,1,Bastedo cared for all the animals that inhabit...
...,...,...
16941,2,The KNVB cup trophy was awarded to Heerenveen....
16942,2,20th Century Fox Television is the syndication...
16943,2,Olivier Rochus always plays with a partner.Oli...
16944,2,Liisi Rist (born 26 June 1991) is an Estonian ...


In [10]:
spark = sparknlp.start()

Exception in thread "main" java.lang.NoSuchMethodError: 'java.lang.Object scala.Some.value()'
	at org.apache.spark.internal.Logging.initializeLogging(Logging.scala:138)
	at org.apache.spark.internal.Logging.initializeLogIfNecessary(Logging.scala:115)
	at org.apache.spark.internal.Logging.initializeLogIfNecessary$(Logging.scala:109)
	at org.apache.spark.deploy.SparkSubmit.initializeLogIfNecessary(SparkSubmit.scala:75)
	at org.apache.spark.deploy.SparkSubmit.doSubmit(SparkSubmit.scala:83)
	at org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1046)
	at org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1055)
	at org.apache.spark.deploy.SparkSubmit.main(SparkSubmit.scala)


RuntimeError: Java gateway process exited before sending its port number

In [ ]:
document_assembler = DocumentAssembler() \
.setInputCol("text") \
.setOutputCol("document") \
.setCleanupMode("shrink")
    
tokenizer = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token") \
.setSplitChars(['-']) \
.setContextChars(['(', ')', '?', '!', '#', '@']) 

normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normalized")\
.setCleanupPatterns(["[^\w\d\s]"]) 

stopwords_cleaner = StopWordsCleaner()\
.setInputCols("normalized")\
.setOutputCol("cleanTokens")\
.setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
.setInputCols(["cleanTokens"]) \
.setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
.setInputCols(["document", 'lemma'])\
.setOutputCol("embeddings")\
.setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
.setInputCols(["document", "embeddings"]) \
.setOutputCol("sentence_embeddings") \
.setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
.setInputCols(["sentence_embeddings"])\
.setOutputCol("class")\
.setLabelColumn("label")\
.setMaxEpochs(5)\
.setLr(0.001)\
.setBatchSize(8)\
.setEnableOutputLogs(True)
#.setOutputLogsPath('logs')

glove_clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

In [ ]:
train_1 = glove_train_r1.dropna(subset=['text', 'label'])
sparkDF_1=spark.createDataFrame(train_1) 
train_2 = glove_train_r2.dropna(subset=['text', 'label'])
sparkDF_2=spark.createDataFrame(train_2) 
train_3 = glove_train_r3.dropna(subset=['text', 'label'])
sparkDF_3=spark.createDataFrame(train_3) 

glove_clf_pipelineModel = glove_clf_pipeline.fit(sparkDF_1)
glove_clf_pipelineModel = glove_clf_pipeline.fit(sparkDF_2)
glove_clf_pipelineModel = glove_clf_pipeline.fit(sparkDF_3)

In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(OUTPUT_DIR)

https://pytorch.org/docs/stable/nn.html#torch.nn.Module

https://huggingface.co/docs/transformers/model_doc/bert

https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/model#transformers.PreTrainedModel

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ef800559-4ff5-4f9a-a563-f7fcfd96a58b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>